# AIStore Python SDK ETL Tutorial

### Set up constants and initialize the client


In [ ]:
from aistore import Client
from aistore.sdk.etl_templates import MD5
from aistore.sdk.etl.webserver.fastapi_server import FastAPIServer
import hashlib
from itertools import cycle

BUCKET_NAME = "bucket-demo"
SPEC_ETL_NAME = "etl-spec-demo"
CODE_ETL_NAME = "etl-code-demo"

# Note: AIS-ETLs require Kubernetes.
client = Client("http://192.168.49.2:8080")
client.bucket(bck_name=BUCKET_NAME).create(exist_ok=True)

### We can initialize ETLs with either [code](https://aiatscale.org/docs/etl#init-code-request) or [spec](https://aiatscale.org/docs/etl#init-spec-request).

#### Initialize an ETL with code:

In [ ]:
# Defining ETL transformation code
import hashlib
from aistore.sdk.etl.webserver.fastapi_server import FastAPIServer

md5_code_etl = client.etl(etl_name=CODE_ETL_NAME)

# Initializing ETL with class using decorator pattern
@md5_code_etl.init_class()
class MD5TransformETL(FastAPIServer):
    def transform(self, data: bytes, _path: str, _etl_args: str) -> bytes:
        """Transform input bytes to MD5 hash"""
        md5 = hashlib.md5()
        md5.update(data)
        return md5.hexdigest().encode()

#### Initialize ETL with spec

In [ ]:
# Use the provided template and substitute in the communication type
template = MD5.format(communication_type="hpush")
md5_spec_etl = client.etl(etl_name=SPEC_ETL_NAME)
md5_spec_etl.init_spec(template=template)

Refer to more ETL templates [here](https://github.com/NVIDIA/aistore/blob/main/python/aistore/sdk/etl).

### List ETLs
Once initialized, we can verify the ETLs are running:

In [ ]:
client.cluster().list_running_etls()

### View ETLs

In [ ]:
md5_code_etl.view()

In [ ]:
md5_spec_etl.view()

## Get an object with ETL transformation applied

### First, create some objects to transform

In [ ]:
import random
import string
import tempfile


def create_and_put_object(
    client: Client,
    bck_name: str,
    obj_name: str,
    provider: str = "ais",
    obj_size: int = 0,
):
    obj_size = obj_size if obj_size else random.randrange(10, 20)
    obj_body = "".join(random.choices(string.ascii_letters, k=obj_size))
    content = obj_body.encode("utf-8")
    with tempfile.NamedTemporaryFile() as file:
        file.write(content)
        file.flush()
        client.bucket(bck_name, provider=provider).object(obj_name).put_file(file.name)
    return content

In [ ]:
content = create_and_put_object(
    client=client, bck_name=BUCKET_NAME, obj_name="object-demo.jpg"
)

### Get single object with ETL code transformation

In [ ]:
client.bucket(BUCKET_NAME).object("object-demo.jpg").get(
    etl_name=md5_code_etl.name
).read_all()

### Get single object with ETL spec transformation

In [ ]:
client.bucket(BUCKET_NAME).object("object-demo.jpg").get(
    etl_name=md5_spec_etl.name
).read_all()

## Transform entire bucket with ETL

In [ ]:
# Create bucket to store transformed objects
dest_bucket = client.bucket("transform-destination-bucket").create(exist_ok=True)

# Transform bucket contents (with on-the-fly object renames)
client.bucket(BUCKET_NAME).transform(
    etl_name=md5_spec_etl.name,
    to_bck=dest_bucket,
    prepend="transformed-",
    ext={"jpg": "txt"},
)

In [ ]:
# Verify rename operations for transformed objects
dest_bucket.list_objects().get_entries()

### Stop ETLs
If an ETL is stopped, any Kubernetes pods created for the ETL are *stopped*, but *not deleted*. Any transforms by the stopped ETL are terminated. Stopped ETLs can be resumed for use with method `start()`:


In [ ]:
md5_code_etl.stop()
md5_spec_etl.stop()
client.cluster().list_running_etls()

### Restart Stopped ETLs

In [ ]:
md5_code_etl.start()
md5_spec_etl.start()
client.cluster().list_running_etls()

### Stop & Delete ETLs
Once completely finished with the ETLs, we clean up (for storage) by stopping the ETLs with `stop` and subsequently deleting the ETLs with `delete`.
Deleting an ETL deletes all pods created by Kubernetes for the ETL as well as any specifications for the ETL on Kubernetes. Consequently, deleted ETLs cannot be started again and will need to be re-initialized.

In [ ]:
md5_code_etl.stop()
md5_spec_etl.stop()

md5_code_etl.delete()
md5_spec_etl.delete()

### Starting Deleted ETL Raises Exception

In [ ]:
md5_code_etl.start()

In [ ]:
md5_spec_etl.start()

### Initialize ETL XOR+Checksum with streaming data

In [ ]:
content = create_and_put_object(
    client=client, bck_name=BUCKET_NAME, obj_name="object-xor-demo.jpg", obj_size=256
)

In [ ]:
class XORChecksumETL(FastAPIServer):
    def transform(self, data: bytes, _path: str, _etl_args: str) -> bytes:
        """Transform data using XOR with key and append MD5 checksum"""
        checksum = hashlib.md5()
        key = b"AISTORE"
        result = bytearray()
        
        # Process data in chunks for streaming-like behavior
        for i in range(0, len(data), 32):
            chunk = data[i:i+32]
            out = bytes([_a ^ _b for _a, _b in zip(chunk, cycle(key))])
            result.extend(out)
            checksum.update(out)
        
        # Append checksum
        result.extend(checksum.hexdigest().encode())
        return bytes(result)

xor_stream_etl = client.etl("xor-md5-stream")
xor_stream_etl.init_class()(XORChecksumETL)

### Get object with XOR+Checksum ETL and verify checksum

In [ ]:
xor_obj = (
    client.bucket(BUCKET_NAME)
    .object("object-xor-demo.jpg")
    .get(etl_name=xor_stream_etl.name)
    .read_all()
)
data, checksum = xor_obj[:-32], xor_obj[-32:]
computed_checksum = hashlib.md5(data).hexdigest().encode()
computed_checksum == checksum

In [ ]:
xor_stream_etl.stop()
xor_stream_etl.delete()

### Cleanup buckets

In [ ]:
for bucket in client.cluster().list_buckets():
    client.bucket(bucket.name).delete()